In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import copy
#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect

import seaborn as sns

from matplotlib import pyplot as plt
init_notebook_mode(connected=True)
cf.go_offline()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer, PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import SelectKBest, VarianceThreshold, RFE, SelectFromModel
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, make_scorer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from joblib import dump, load
from Logger import RegressionLogger, FuncTransformer, ColSelect
import copy

pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

In [2]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")
df.drop(["Institution Name", "Unnamed: 0"], axis=1, inplace=True)
X_fe = load("objects/features.joblib")
y = load("objects/y.joblib")

In [3]:
model_log = load("objects/model_logging.joblib")

In [4]:
xgb_model = model_log[6]["model"]

# More Feature Engineering

## Impute X_fe

In [5]:
X_fe

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und

In [6]:
imputer = KNNImputer()

In [7]:
X_df = pd.DataFrame(imputer.fit_transform(X_fe), columns=X_fe.columns, index=X_fe.index)

In [8]:
X_df.head()

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und

## Multivariate Plots

In [9]:
price_names = X_df.filter(regex="price").columns

sector_names = X_df.filter(regex="Sector").columns
state_names = X_df.filter(regex="State").columns
hbcu = X_df.filter(regex="Historically").columns[0]



In [10]:
bins = [X_df[price_names[2]].quantile(0.20 * i) for i in range (0,6)]
bins[5]
labels = [1, 2, 3, 4, 5]

In [11]:
binned_price = pd.cut(X_df[price_names[2]], 5, labels=labels).astype(int)

In [12]:
binned_log_core_revenues = binned_price * X_df["Log_Core_Revenues"]

In [13]:
sector_names

Index(['Sector of institution (HD2018)_Private for-profit, 4-year or above',
       'Sector of institution (HD2018)_Private for-profit, less-than 2-year',
       'Sector of institution (HD2018)_Private not-for-profit, 2-year',
       'Sector of institution (HD2018)_Private not-for-profit, 4-year or above',
       'Sector of institution (HD2018)_Public, 2-year',
       'Sector of institution (HD2018)_Public, 4-year or above',
       'Sector of institution (HD2018)_isMissing'],
      dtype='object')

In [14]:
range_ = [12, 26]
degree = wg.IntSlider(value=1, min=1, max=3)
feature_picker = wg.Dropdown(
    options=X_df.columns,
    value=X_df.columns[0],
    description='Feature:',
    disabled=False,
)
color_picker = wg.Dropdown(
    options=X_df.filter(regex="Sector|Historically|State|Basic|Size and Setting|category").columns,
    value=X_df.filter(regex="Sector|Historically|State|Basic|Size and Setting|category").columns[0],
    description='Color:',
    disabled=False,
)


def fe_plot(feature, color):
    x_var = X_df[feature]
    
    fig2 = px.scatter(
        x=x_var, y=y, 
        color=X_df[color].astype("category").replace({1.0:'yes', 0.0:'no'}),
    )
    
    fig2.add_trace(go.Scatter(
        x=[50], y=[50], mode="markers"
    ))
    
    fig2.update_layout(
        title="Multi Variate Plots",
        xaxis={
            "title": feature,
            "range": [X_df[feature].min() - X_df[feature].std() / 10, X_df[feature].max() + X_df[feature].std() / 10]
        },
        yaxis={
            "title": "Graduation Rate",
            "range": [-5,105]
        },
        legend={
            'orientation': 'v',
            'title': color,
            'x': 0.5,
            'y': 1.1,
            'bgcolor': "rgba(176,196,222,0.7)",
            'bordercolor': "black",
            'borderwidth': 2,
            'itemclick': "toggle",
        },
        font=dict(
            family="Courier New, monospace",
            size=16),
        height=700
    )

    fig2.update_traces(marker=dict(size=12,
                                  line=dict(width=2,
                                  color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    
    
    
    
    display(fig2.show())
    

ui2 = wg.HBox([feature_picker, color_picker])  

func_params = {
    'feature': feature_picker,
    'color': color_picker
}


output = wg.interactive_output(fe_plot, func_params)   


display(ui2, output)   
    

Output()

In [15]:
# Highly Residential Columns
private_non_profit = X_df.filter(regex="Sector").iloc[:, [2,3,6]].sum(axis=1)
rev_by_sector = private_non_profit * X_df["Log_Core_Revenues"]

In [16]:
primarily_nonresidential = X_df.filter(regex="primarily nonresidential").sum(axis=1)
primarily_residential = X_df.filter(regex="primarily residential").sum(axis=1) * 2
primarily_highly_residential = X_df.filter(regex="highly residential").sum(axis=1) * 3

In [17]:
engineered_features = pd.concat([private_non_profit, rev_by_sector], axis=1)

In [18]:
multiplier = wg.FloatSlider(value=0.5, max=1, min=0, step=0.01, continuous_update=False)
caption = wg.Label(value=f"y = M X + B")
ui = wg.HBox([multiplier, caption])

def fe_plot(slope):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=X_df["Log_Core_Revenues"] + private_non_profit * slope * X_df["Log_Core_Revenues"],
        y=y, mode="markers"
    ))
    
    
    display(fig.show())
    
out = wg.interactive_output(fe_plot, {'slope': multiplier})   
display(ui, out)

Output()

## Engineered Features

<ul>
    <li>Institutional Size Category as ordinal</li>
    <li>Residential Level as ordinal</li>
    <li>Institutional size category * Residential Leve</li>
    <li>Log Instructional Staff</li>
    <li>Log White Men Instructional Staff</li>
</ul>

In [19]:
private_non_profit = X_df.filter(regex="Sector").iloc[:, [2,3,6]].sum(axis=1)

In [20]:
 X_df.filter(regex="Sector").iloc[:, [2,3,6]]

,"Sector of institution (HD2018)_Private not-for-profit, 2-year","Sector of institution (HD2018)_Private not-for-profit, 4-year or above",Sector of institution (HD2018)_isMissing
UnitID,,,
180203,0.0,0.0,0.0
222178,0.0,1.0,0.0
138558,0.0,0.0,0.0
172866,0.0,0.0,0.0
108232,0.0,0.0,0.0
...,...,...,...
217059,0.0,1.0,0.0
141361,0.0,1.0,0.0
206695,0.0,0.0,0.0


In [21]:
cc_ss = df.filter(regex="Size and Setting").columns[0]
engineered_features["CC_Residential_Level"] = df[cc_ss].map({
    'Two-year, very small': 1,
    'Two-year, small': 1,
    'Two-year, medium': 1,
    'Two-year, large': 1,
    'Two-year, very large': 1,
    
    'Four-year, very small, primarily nonresidential': 1,
    'Four-year, very small, primarily residential': 2,
    'Four-year, very small, highly residential': 3,
    
    'Four-year, small, primarily nonresidential': 1,
    'Four-year, small, primarily residential': 2,
    'Four-year, small, highly residential': 3,
    
    'Four-year, medium, primarily nonresidential': 1,
    'Four-year, medium, primarily residential': 2,
    'Four-year, medium, highly residential': 3,
    
    
    'Four-year, large, primarily nonresidential': 1,
    'Four-year, large, primarily residential': 2,
    'Four-year, large, highly residential': 3,
    
    'Not applicable, not in Carnegie universe (not accredited or nondegree-granting)': 2,
    'Exclusively graduate/professional': 1,
    'isMissing': 2
})
engineered_features["Institution_Size_Category"] = df["Institution size category (HD2018)"].map({
    'Under 1,000': 1,
    '1,000 - 4,999': 2,
    '5,000 - 9,999': 3,
    '10,000 - 19,999': 4,
    '20,000 and above': 5,
    'isMissing': 1    
})

engineered_features["price*percentage of women Under 25"] = X_df["Total women (EF2013B_RV  Undergraduate  Age under 25 total)_as_percentage"]\
    * X_df["Total price for out-of-state students living off campus (with family)  2012-13 (DRVIC2012_RV)"]
engineered_features["Residential*Size"] = engineered_features["CC_Residential_Level"] * engineered_features["Institution_Size_Category"]
engineered_features["Log_Total_Instructional_Staff"] = np.log(X_df["Grand total instructional_staff"])
engineered_features["Log_White_Men_Instructional_Staff"] = np.log(X_df["White men instructional_staff"] + 1)

engineered_features["SAT_Math * Price_off_campus_with_family"] = X_df["SAT Math 75th percentile score (IC2012_RV)"] * X_df["Total price for out-of-state students living off campus (with family)  2012-13 (DRVIC2012_RV)"]

engineered_features["Percentage_of_women_under_25 * SAT_Math"] = X_df["Total women (EF2013B_RV  Undergraduate  Age under 25 total)_as_percentage"] * X_df["SAT Math 75th percentile score (IC2012_RV)"]

engineered_features["Log_Core_Revenues_adj_by_non_profit "] = X_df["Log_Core_Revenues"] + private_non_profit * 0.12 * X_df["Log_Core_Revenues"]

In [22]:
sector_names = X_df.filter(regex="Sector").columns
state_names = X_df.filter(regex="State").columns

In [23]:
engineered_features = engineered_features.rename({0: "Private_Non_Profit", 1: "revenue_by_sector"}, axis=1)

# Optimizing new Model

In [24]:
def weight(x, thresh1, thresh2, weight):
    if x <= thresh1:
        return 1
    elif x >= thresh2:
        return 1
    
    else:
        return weight
   

## Poly Features to create features that are even more correlated with Graduation Rates

In [25]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('preprocess', PolynomialFeatures(degree=3)),
    ('classifier', RandomForestRegressor())
])

In [26]:
engineered_data = pipeline['preprocess'].fit_transform(engineered_features)

In [27]:
poly_feat_names = pipeline['preprocess'].get_feature_names(engineered_features.columns)

In [28]:
poly_df = pd.DataFrame(engineered_data, columns=poly_feat_names)

In [29]:
poly_df.index = y.index

In [30]:
for i in poly_df.columns:
    corr = poly_df[i].corr(y)
    
    if corr > 0.63:
        pass
        # engineered_features[i] = poly_df[i]

In [31]:
engineered_features.head(3)

,Private_Non_Profit,revenue_by_sector,CC_Residential_Level,Institution_Size_Category,price*percentage of women Under 25,Residential*Size,Log_Total_Instructional_Staff,Log_White_Men_Instructional_Staff,SAT_Math * Price_off_campus_with_family,Percentage_of_women_under_25 * SAT_Math,Log_Core_Revenues_adj_by_non_profit
UnitID,,,,,,,,,,,
180203,0.0,0.000000,1,1,2212.60,1,2.564949,1.386294,4663480.0,142.48,15.772637
222178,1.0,18.730952,3,3,27500.00,9,5.579730,4.882802,18750000.0,528.00,20.978666
138558,0.0,0.000000,2,2,7754.37,4,5.003946,4.143135,5388630.0,241.90,18.071008


In [32]:
df_total = pd.concat([X_df, engineered_features], axis=1)

In [33]:
df_total.head(3)

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und

# Feature Selection

## Train XGB Model on all of our features

In [34]:
params = {
    
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'n_estimators': 100,
    
    'max_depth': 19,
    
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'colsample_bynode': 0.9,
    
    'gamma': 0.7,
    'alpha': 0.8,
    
    'min_child_weight': 7,
    
    'importance_type': 'gain'
}

In [35]:
xgb_pipeline = xgb_model

In [36]:
logger = RegressionLogger(params)

In [37]:
logger.train_update(xgb_pipeline, df_total, y)

In [38]:
logger.mae

8.169575929641724

In [39]:
feature_importances = xgb_pipeline['classifier'].feature_importances_

importance_map = dict(zip(df_total.columns.to_list(), feature_importances))
to_drop = list({i: v for i, v in importance_map.items() if v == 0}.keys())
importance_map = {i: v for i, v in importance_map.items() if v != 0}

In [40]:
top_features = df_total.drop(to_drop, axis=1)

In [41]:
logger.train_update(xgb_pipeline, top_features, y)

In [42]:
logger.mae

8.176128518992456

In [43]:
feature_importances2 = xgb_pipeline['classifier'].feature_importances_

importance_map2 = dict(zip(top_features.columns.to_list(), feature_importances))
to_drop2 = list({i: v for i, v in importance_map2.items() if v == 0}.keys())
importance_map2 = {i: v for i, v in importance_map2.items() if v != 0}

top_features2 = top_features.drop(to_drop2, axis=1)

logger.train_update(xgb_pipeline, top_features2, y)

logger.mae

8.04765639633968

In [44]:
feature_importances3 = xgb_pipeline['classifier'].feature_importances_

importance_map3 = dict(zip(top_features2.columns.to_list(), feature_importances))
to_drop3 = list({i: v for i, v in importance_map3.items() if v == 0}.keys())
importance_map3 = {i: v for i, v in importance_map3.items() if v != 0}

top_features3 = top_features2.drop(to_drop3, axis=1)

logger.train_update(xgb_pipeline, top_features3, y)

logger.mae

8.241408109664917

In [45]:
feature_importances4 = xgb_pipeline['classifier'].feature_importances_

importance_map4 = dict(zip(top_features3.columns.to_list(), feature_importances))
to_drop4 = list({i: v for i, v in importance_map4.items() if v == 0}.keys())
importance_map4 = {i: v for i, v in importance_map4.items() if v != 0}

top_features4 = top_features3.drop(to_drop4, axis=1)

logger.train_update(xgb_pipeline, top_features4, y)

logger.mae

8.006410643972199

## Creating Our new Holdout set

In [46]:
df = pd.read_csv("data/cleaned_df.csv", index_col="Institution Name")

sample_schools = df.filter(regex=r'(Asheville|Spelman|University of Delaware)', axis=0)["UnitID"]

index = df.filter(regex=r'(Asheville|Spelman|University of Delaware)', axis=0).index

sample_schools_x = top_features2.loc[sample_schools]
sample_schools_y = y.loc[sample_schools]
sample_schools_x.index = index
sample_schools_y.index = index

top_features2.drop(sample_schools, inplace=True)
y.drop(sample_schools, inplace=True)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(top_features2, y, test_size=0.1, random_state=0)

list_ = [x_train, x_test, y_train, y_test, sample_schools_x, sample_schools_y]

In [48]:
model_params = params

In [49]:
model_params["model_name"] = "XGBRegressor"
model_params["model"] = xgb_pipeline

In [50]:
logger.__init__(model_params)
logger.train_holdout(xgb_pipeline, x_train, x_test, y_train, y_test)

In [51]:
logger.save_log("Final Feature Engineering")
model_log.append(logger.record())
model_log[-1]["model"] = xgb_pipeline

In [52]:
dump(list_, "objects/holdout_total.joblib")

['objects/holdout_total.joblib']

In [53]:
logger.mae

7.532104040014333

**With Feature Engineering Steps our Model predicts with 7.5 Mean Absolute Error**

## Last Hyperparameter Tuning

In [54]:
xgb_pipeline['classifier'].get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 0.9,
 'colsample_bytree': 0.8,
 'gamma': 0.7,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.07,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 10,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 200,
 'n_jobs': 4,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0.899999976,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'alpha': 0.9}

In [55]:
final_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.07,
    'n_estimators': 200,
    
    'max_depth': 7,
    
    'colsample_bytree': 0.8,
    'colsample_bylevel': 1,
    'colsample_bynode': 0.9,
    
    'gamma': 0.7,
    'alpha': 0.9,
    
    'min_child_weight': 10,
    
    'importance_type': 'gain'
}
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', xgb.XGBRegressor(**final_params))
])
logger.__init__(final_params)

In [56]:
_ = xgb_pipeline.fit(x_train, y_train)

In [57]:
y_pred = xgb_pipeline.predict(x_test)

In [58]:
mean_absolute_error(y_pred, y_test)

7.532104040014333

In [59]:
logger.hyperparams["model_name"] = "XGBRegressor"
logger.hyperparams["model"] = xgb_model

In [60]:
logger.train_holdout(xgb_pipeline, x_train, x_test, y_train, y_test)

In [61]:
logger.mae

7.532104040014333

In [62]:
logger.save_log("Final Tuned Model")
model_log.append(logger.record())
model_log[-1]["model"] = xgb_pipeline

In [63]:
pd.DataFrame().from_dict(model_log)

,model,model_name,alpha,l1_ratio,rsquared,rmse,mae,train_rsquared,train_rmse,train_mae,rsquared_black,rmse_black,mae_black,train_rsquared_black,train_rmse_black,train_mae_black,notes,Imputer,Scaler,modelimputer,polyfeatuers,n_estimators,max_depth,min_samples_leaf,objective,learning_rate,colsample_bytree,colsample_bynode,gamma,min_child_weight,importance_type,colsample_bylevel
0,"(ColumnTransformer(remainder='passthrough',\n ...",Elastic Net,0.1,0.5,0.559300,14.215243,9.888576,0.584740,14.147507,9.992870,0.376473,20.273454,14.861809,0.4866,18.537008,13.443723,Baseline Model,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.1,0.5,0.562076,13.626806,9.479094,0.590908,14.080223,9.889907,NaN,NaN,NaN,NaN,NaN,NaN,KNN Imputation,KNNImputer,Standard Scaler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.571013,13.487057,9.450582,0.604599,13.842609,9.720690,NaN,NaN,NaN,NaN,NaN,NaN,Polynomial features added,NaN,NaN,KNNImputer,Polynomial Features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.565345,13.575853,9.430176,0.600432,13.915353,9.784322,NaN,NaN,NaN,NaN,NaN,NaN,top 250 polyfeatures used,NaN,NaN,KNNImputer,Polynomial Features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.3,0.8,0.536707,14.015952,9.509088,0.634621,13.306718,9.332271,NaN,NaN,NaN,NaN,NaN,NaN,Tuned Elstic Net Model,KNNImputer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(FeatureUnion(transformer_list=[('numeric',\n ...",Random Forest,NaN,NaN,0.661840,11.974458,8.224307,0.908428,6.661632,4.355189,NaN,NaN,NaN,NaN,NaN,NaN,Tuned Random Forest,KNNImputer,NaN,NaN,NaN,150.0,15.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.9,NaN,0.647749,12.221397,8.214601,0.978999,3.190186,2.344896,NaN,NaN,NaN,NaN,NaN,NaN,Tuned XGB Model,KNNImputer,NaN,NaN,NaN,200.0,7.0,NaN,reg:squarederror,0.07,0.8,0.9,0.7,10.0,gain,NaN
7,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Feature Engineering,NaN,NaN,NaN,NaN,100.0,19.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9
8,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Tuned Model,NaN,NaN,NaN,NaN,200.0,20.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9
9,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Feature Engineering,NaN,NaN,NaN,NaN,100.0,19.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9


In [64]:
dump(top_features2, "objects/engineered_features.joblib")

['objects/engineered_features.joblib']

In [66]:
dump(model_log, "objects/model_logging.joblib")

['objects/model_logging.joblib']

In [68]:
pd.DataFrame().from_dict(model_log)

,model,model_name,alpha,l1_ratio,rsquared,rmse,mae,train_rsquared,train_rmse,train_mae,rsquared_black,rmse_black,mae_black,train_rsquared_black,train_rmse_black,train_mae_black,notes,Imputer,Scaler,modelimputer,polyfeatuers,n_estimators,max_depth,min_samples_leaf,objective,learning_rate,colsample_bytree,colsample_bynode,gamma,min_child_weight,importance_type,colsample_bylevel
0,"(ColumnTransformer(remainder='passthrough',\n ...",Elastic Net,0.1,0.5,0.559300,14.215243,9.888576,0.584740,14.147507,9.992870,0.376473,20.273454,14.861809,0.4866,18.537008,13.443723,Baseline Model,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.1,0.5,0.562076,13.626806,9.479094,0.590908,14.080223,9.889907,NaN,NaN,NaN,NaN,NaN,NaN,KNN Imputation,KNNImputer,Standard Scaler,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.571013,13.487057,9.450582,0.604599,13.842609,9.720690,NaN,NaN,NaN,NaN,NaN,NaN,Polynomial features added,NaN,NaN,KNNImputer,Polynomial Features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,NaN,NaN,0.565345,13.575853,9.430176,0.600432,13.915353,9.784322,NaN,NaN,NaN,NaN,NaN,NaN,top 250 polyfeatures used,NaN,NaN,KNNImputer,Polynomial Features,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(FeatureUnion(transformer_list=[('numeric',\n ...",Elastic Net,0.3,0.8,0.536707,14.015952,9.509088,0.634621,13.306718,9.332271,NaN,NaN,NaN,NaN,NaN,NaN,Tuned Elstic Net Model,KNNImputer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"(FeatureUnion(transformer_list=[('numeric',\n ...",Random Forest,NaN,NaN,0.661840,11.974458,8.224307,0.908428,6.661632,4.355189,NaN,NaN,NaN,NaN,NaN,NaN,Tuned Random Forest,KNNImputer,NaN,NaN,NaN,150.0,15.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.9,NaN,0.647749,12.221397,8.214601,0.978999,3.190186,2.344896,NaN,NaN,NaN,NaN,NaN,NaN,Tuned XGB Model,KNNImputer,NaN,NaN,NaN,200.0,7.0,NaN,reg:squarederror,0.07,0.8,0.9,0.7,10.0,gain,NaN
7,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Feature Engineering,NaN,NaN,NaN,NaN,100.0,19.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9
8,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Tuned Model,NaN,NaN,NaN,NaN,200.0,20.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9
9,"(FeatureUnion(transformer_list=[('numeric',\n ...",XGBRegressor,0.8,NaN,0.709581,11.073809,7.532104,0.984606,2.732257,2.017823,NaN,NaN,NaN,NaN,NaN,NaN,Final Feature Engineering,NaN,NaN,NaN,NaN,100.0,19.0,NaN,reg:squarederror,0.10,0.8,0.9,0.7,7.0,gain,0.9
